In [ ]:
# -----------------------------------------------------------------------------------
# 파일명       : GRU_AutoEncoder.ipynb
# 설명         : 원격 검침 데이터의 이상패턴을 파악하기 위한 AutoEncoder 모델 학습                
# 작성자       : 이민하
# 작성일       : 2024-11-27
# 
# 사용 모듈    :
# - sys, os                          # 시스템 설정 및 경로 관리
# - GRUAutoEncoderModule             # GRU 기반 AutoEncoder 모델 정의 및 학습 함수 모듈
# - torch, torch.nn, F               # PyTorch 모델 구축 및 연산
# - torch.utils.data                 # 데이터셋 및 데이터로더 처리
# - pickle                           # 객체 저장 및 로딩 (직렬화)
# - pandas                           # 데이터프레임 기반 데이터 처리
# - sklearn.model_selection          # 학습/검증용 데이터 분할
# - sklearn.preprocessing            # 데이터 정규화 및 스케일링
# - torch.optim, lr_scheduler        # 최적화 및 학습률 조정
# - torchsummary                     # 모델 구조 요약 출력
# - torchmetrics.regression          # 회귀 모델 평가 지표 계산
# - matplotlib.pyplot                # 시각화 및 그래프 출력
#
# -----------------------------------------------------------------------------------
# >> 주요 기능
# - 데이터 및 모델 불러오기
# - 모델 학습
#
# >> 업데이트 내역
# [2024-11-27] MLP 구조의 Vanilla AutoEncoder 모델 학습
# [2024-11-30] VAE AutoEncoder 모델 학습 (성능 저하)
# [2024-12-03] Recurrent AutoEncoder 모델 학습 (성능 저하)
# [2024-12-09] GRU AutoEncoder 모델 학습
#
# >> 성능
# 개선 전 :
#   Vanilla AutoEncoder
#   수도 Custom Loss : 5.33, 전기 Custom Loss : 4.69
# 
# 개선 후 :
#   GRU AutoEncoder
#   수도 Custom Loss : 1.07, 전기 Custom Loss : 1.03
# -----------------------------------------------------------------------------------


In [ ]:
# 시스템 설정 및 경로 관리
import sys
import os

# GRU 기반 AutoEncoder 모델 정의 및 학습 함수 모듈
from GRUAutoEncoderModule import *

# PyTorch 모델 구축 및 연산
import torch
import torch.nn as nn
import torch.nn.functional as F

# 데이터셋 및 데이터로더 처리
from torch.utils.data import Dataset, DataLoader

# 객체 저장 및 로딩
import pickle

# 데이터프레임 기반 데이터 처리
import pandas as pd

# 학습/검증용 데이터 분할
from sklearn.model_selection import train_test_split

# 데이터 정규화 및 스케일링
from sklearn.preprocessing import MinMaxScaler, RobustScaler

# 최적화 및 학습률 조정
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

# 모델 구조 요약 출력
from torchsummary import summary

# 회귀 모델 평가 지표 계산
from torchmetrics.regression import R2Score, MeanAbsoluteError, MeanAbsolutePercentageError, MeanSquaredError

# 시각화 및 그래프 출력
import matplotlib.pyplot as plt

In [ ]:
# 데이터 경로 설정
DATA_PATH = '../Data/'

# csv 파일 -> DataFrame 형태로 불러오기
electric_df = pd.read_csv(DATA_PATH + 'electric_df_clear_2.csv', index_col = 0)
water_df = pd.read_csv(DATA_PATH + 'water_diff_df_clear.csv') # 수도 데이터는 변화량 데이터 사용

In [ ]:
# 데이터 확인
print(electric_df.head())
print()
print(water_df.head())

               0     1     2     3     4     5     6     7     8     9  ...  \
Unnamed: 0                                                              ...   
0           1.23  1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  ...   
1           1.45  1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  ...   
2           1.73  1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  ...   
3           1.62  1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  ...   
4           1.31  1.48  1.74  1.71  1.30  1.49  1.94  1.68  1.26  1.39  ...   

              18    19    20    21    22    23    24    25    26    27  
Unnamed: 0                                                              
0           1.77  1.68  1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  
1           1.68  1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  
2           1.20  1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  
3           1.42  1.88  1.67  1.24  1.39  1.67  1.60  1.26  1.41  1.68  
4       

In [ ]:
# 학습 파라미터 설정
EPOCH = 10000
BATCH_SIZE = 64
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [ ]:
# 데이터셋, 데이터로더 변환
electric_trainDS = CustomDataset(electric_df)
water_trainDS = CustomDataset(water_df)

electric_trainDL = DataLoader(electric_trainDS, batch_size = BATCH_SIZE)
water_trainDL = DataLoader(water_trainDS, batch_size = BATCH_SIZE)

In [ ]:
# 모델 파라미터 설정
input_size = 28
latent_dim = 16
n_layers = 2

# 학습 Custom Loss 파라미터 설정
penalty = 1
threshold = 0.02

# 모델 생성
model = GRUAutoEncoderModel(input_size = input_size, latent_dim = latent_dim, n_layers = n_layers)

# 옵티마이저 생성
optimizer = optim.Adam(model.parameters(), lr = LR)

# Learning Rate Scheduler 생성
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode = 'min', patience = 10, verbose = True)


c:\Users\KDP-2\anaconda3\envs\Project_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# 모델 구조 출력
summary(model, input_size = (28, ))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
               GRU-1  [[-1, 1, 16], [-1, 2, 16]]               0
               GRU-2       [[-1, 28], [-1, 28]]               0
            Linear-3                   [-1, 28]             812
Total params: 812
Trainable params: 812
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [ ]:
# 모델 학습
encoder_loss = training(model, electric_trainDL,
                    optimizer, penalty, threshold, EPOCH, scheduler, DEVICE)

[1 / 10000]
- TRAIN LOSS : 1.0788606093568
tensor([1.4200, 1.4800, 1.6400, 1.5300, 1.4600, 1.4400, 1.6300, 1.5400, 1.4600,
        1.4700, 1.5900, 1.3700, 1.3300, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400,
        1.6400, 1.5100, 1.4600, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400, 1.6400,
        1.5100])
tensor([1.4159, 1.4399, 1.6666, 1.5920, 1.4378, 1.4351, 1.6259, 1.5464, 1.5205,
        1.5719, 1.5760, 1.3710, 1.3746, 1.3505, 1.4539, 1.6719, 1.6483, 1.4497,
        1.4623, 1.5375, 1.4297, 1.3543, 1.5570, 1.7565, 1.6848, 1.5037, 1.4958,
        1.5806], grad_fn=<SelectBackward0>)
[2 / 10000]
- TRAIN LOSS : 1.0789204407397963
tensor([1.4200, 1.4800, 1.6400, 1.5300, 1.4600, 1.4400, 1.6300, 1.5400, 1.4600,
        1.4700, 1.5900, 1.3700, 1.3300, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400,
        1.6400, 1.5100, 1.4600, 1.3600, 1.4500, 1.6900, 1.7900, 1.4400, 1.6400,
        1.5100])
tensor([1.4159, 1.4399, 1.6666, 1.5920, 1.4378, 1.4351, 1.6259, 1.5464, 1.5205,
        1.5719, 1.5760, 1.3710, 1